# Setting server asset tag

<div class="alert alert-block alert-info" style="background-color:">
 Welcome to <b>python-iLOrest-library</b> <br> This is an example of <b>setting server asset tag.</b> <br> As you are here let's get it done easily.<br>
    To perform this you need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library" style = "background-color:white">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

 <div class="alert alert-block alert-success">
    <h1> 1. Below we are importing all the necessary modules from the ilorest library. </h1>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    
</div>

In [ ]:
import sys
import json
import time
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

<div class = "alert alert-block alert-success"><h1>3. This is our set_server_asset_tag function overview. </h1>
    <br>
1. The <b>set_server_asset_tag</b> function sets the server asset tag on an iLO device. The function takes two parameters: _redfishobj and tag. 
    <ul>
    <li><b>_redfishobj</b> is a Redfish client object that represents the connection to the server</li>
    <li><b>tag </b> is a string that represents the asset tag to be set for the server. </li>
    </ul><br>
2. In the <b>set_server_asset_tag</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using get_resource_directory module.<br><br>
3. In the ILO rest api the information about server asset tag is stored inside the <b>systems json object</b>.
    And now through system_response variable,        we are making a http <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods" style="background-color:powderblue;">GET operation</a> to and trying to obtain the server asset tag of iLO system.<br><br>
4. In the system members response object with different <b>odata_id</b> we get a system uri .
    Then again an http GET request is made and response is stored in <b>system_response</b>.
    Then we set the system_members_uri.
    <br><br>
5. Else if resource directory is enabled then we will simply iterate through all instances inside resources 
    and if #ComputerSystem attribute is present in <b>@odata.type</b> attribute then we'll set <b>systems_members_uri</b> and <i> get </i> <b>systems_members_response</b> of the user through redfish object.<br><br>  
6.The function then calls the <b>patch()</b> method to update the system's AssetTag property with the new tag provided as input to the function.<br><br> 
7. If the iLO device responds with a status code outside of 200 or 201, the function attempts to access the iLO extended message info to see what went wrong.<br><br>   
8. If the patch() operation is successful, the updated asset tag is printed to the console. The function then waits for 10 seconds before retrieving and displaying the updated asset tag.<br><br>
</div>

In [ ]:
def set_server_asset_tag(_redfishobj, tag):

    systems_members_uri = None
    systems_members_response = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        systems_members_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_members_response = _redfishobj.get(systems_members_uri)
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#ComputerSystem.' in instance['@odata.type']:
                systems_members_uri = instance['@odata.id']
                systems_members_response = _redfishobj.get(systems_members_uri)

    if systems_members_response and systems_members_uri and tag:
        print("Current Asset Tag: \'%s\'\n" % systems_members_response.dict.get("AssetTag"))

        resp = _redfishobj.patch(systems_members_uri, {"AssetTag" : tag})
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))
            time.sleep(10) #going to wait 10 seconds before obtaining the LED indicator state
            sys.stdout.write("\nUpdated Asset Tag: \'%s\'\n" % _redfishobj.\
                                                    get(systems_members_uri).dict.get("AssetTag"))

<div class="alert alert-block alert-success"><h1 >3. This is Main function overview.</h1><br>
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE TO PROCEED FURTHER<br>
    <ul>
        <li>Once your credentials are entered a redfish object is created using the input system URL, login account, and login password with the help of RedfishClient class and a session is created and now you will be logged in to the server.</li>
        <li>If server is down or unreachable we get server not reachable message.</li>
        <li>If not then we make a call to <b>set_server_asset_tag</b> to set server asset tag of system.</li>
        <li>After getting required information <i><b>Redfish.logout()</b></i> closses the particular session and logs us out of the system.</li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    #asset tag property in computer systems
    ASSET_TAG = "Asset01"
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = True

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    set_server_asset_tag(REDFISHOBJ, ASSET_TAG)
    REDFISHOBJ.logout()


<div class = "alert alert-block alert-warning">
    <b>When running on the server locally use the following commented values</b><br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

<div class = "alert alert-block alert-success">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>